In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def detectar(net):
  layer_names = net.getLayerNames()
  output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
  return output_layers

In [3]:
def dibujar(img, objId, prob, x, y, xw, yh):
    label= str(clases[objId])
    color=COLORES[objId]
    cv2.rectangle(img, (x,y), (xw,yh), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [4]:
file="luna.jpg"
config="yolov3.cfg"
txt="yolov3.txt"
pesos="yolov3.weights"
scale=0.00392

# leer la lista de clases de objetos a buscar 
clases = None
with open(txt, 'r') as f:
    clases = [line.strip() for line in f.readlines()]
COLORES = np.random.uniform(0, 255, size=(len(clases), 3))
    
# obtener datos de la imagen
image= cv2.imread(file)
ancho = image.shape[1]
alto = image.shape[0]

# valores del umbral
conf_threshold = 0.5
nms_threshold = 0.4

# arrays para cargar datos
arrayId=[]
arrayProb=[]
arrayCajas=[]

# crear el DL
dl= cv2.dnn.readNet(pesos, config)
blob=cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

# detectar
dl.setInput(blob)
objetos= dl.forward(detectar(dl))

# recorrer posibles objetos
for objeto in objetos:
    for algo in objeto:
        objValor=algo[5:]
        objId=np.argmax(objValor)
        objProb=objValor[objId]
        if objProb > 0.5:
            centro_x= int(algo[0] * ancho)
            centro_y= int(algo[1] * alto)
            w=int(algo[2]*ancho)
            h=int(algo[3]*alto)
            x=centro_x- w/2 
            y=centro_y- h/2
            arrayId.append(objId)
            arrayProb.append(float(objProb))
            arrayCajas.append([x,y,w,h])
indices= cv2.dnn.NMSBoxes(arrayCajas, arrayProb, conf_threshold, nms_threshold)

for i in indices:
  i= i[0]
  caja=arrayCajas[i]
  x=caja[0]
  y=caja[1]
  w=caja[2]
  h=caja[3]
  dibujar(image, arrayId[i], arrayProb[i], round(x), round(y), round(x+w), round(y+h))
 
cv2.imwrite("resultado.jpg", image)
print("Listo")

Listo
